### O.6 공공데이터

공공데이터는 정부에서 포털형식으로 제공되고 있다. 교통, 재난, 의료 등 기관별로 제공하는 공공데이터를 묶어서 포털로 제공하고 있다.

또한 시군구별로 제공하는 열린데이터가 있다. 그렇기 때문에 data.시군구.go.kr과 같이 url이 명명되어 있다. url은 요청인자를 슬래시로 구분하여 적는다. 반면에 공공데이터는 data.go.kr로 명명된다. 공공데이터는 url에 요청인자를 &로 구분하여 적고 있다.



### O.6.1 인증키
####  서울시 열린데이터 인증키 신청
열린데이터를 이용하기 위해서는 회원가입 후, 인증키를 발급받아야 한다. Home > 나의화면 > 인증키 신청으로 페이지를 이동해서, 인증키를 신청하고 발급한다. url은 이용하실 열린데이터광장 서비스중 URL 하나를 적어주면 된다.


## URL 인코딩
URL은 128 ($2^7$) ASCII 문자만 받도록 되어있다. 영문자, 숫자, -   _   .   ~는 그대로 사용하지만 !    #   $   &   '   (   )   *   +   ,   /   :   ;   =   ?   @   [   ]는 퍼센트 인코딩 (또는 url 인코딩)을 한다.

URL 인코딩은 온라인 사이트 https://www.urlencoder.org/ 에서 확인할 수 있다.

url encoding은 %를 맨 앞에 그리고 16진수 2자리를 덧붙인다. #은 %23으로 변환된다. 아래에서 보듯이 한글은 ASCII 128 문자에 포함되지 않으므로, 퍼센트 인코딩이 된다. Python3에서는 urlparse(python2) 대신, urllib.parse를 사용한다.

In [59]:
import urllib

print (urllib.parse.urlencode({'#q':'한글', 'dd':'dd'}))
print (urllib.parse.urlencode({'#q':u'한글'.encode('utf-8')}))

%23q=%ED%95%9C%EA%B8%80&dd=dd
%23q=%ED%95%9C%EA%B8%80


위: 한개의 글자에 3개의 인코딩이 들어감 

인증키를 살펴보면, '%'가 포함되어 있을 수 있다. '%'문자를 url encoding하는 경우 '%'를 '%25'로 encoding한다.

In [2]:
import urllib

params=urllib.parse.urlencode({'key':'%3D'})
print (params)

key=%253D


% -> %25로 인코딩댐


### 인증키 파일 작성
일단 인증키를 파일에 저장하자. '비밀번호'와 같은 보안내용이 프로그램에 직접 사용되지 않도록 주의한다. 'key.properties' 파일에 Property형식에 따라 key, value로 저장한다. 단 윈도우 메모장에 붙여넣으면 BOM 문자가 문제를 일으킨다. 공공데이터포털 utf-8로 변환된 키 값은 urlencode()를 하지 않는다. 파일에 아래와 같이 적어 넣는다.

-> 열린데이터

dataseoul=73725...

->공공데이터

gokr=8Bx4C1%2B...

#### 인증키 라이브러리
자신의 라이브러리를 만들어보자 (참조: how_to_import.ipynb)


* 디렉토리에 init.py 파일을 넣는다.

* 디렉토리 명이 패키지가 되고, import를 사용할 수 있다.

* mylib.py에는 getKey()와 테스트 용도로 sayHello()를 추가한다.
* 
디렉토리의 파일 구조는:

src/

    __init__.py
    
    mylib.py
    
    key.properties

In [5]:
# %%writefile src/key.properties
# dataseoul=73725...
# gokr=8Bx4C1%2B...

In [3]:
%%writefile src/mylib.py
def sayHello():
    print("hello")

def getKey(keyPath):
    d=dict()
    f=open(keyPath,'r')
    for line in f.readlines():
        row=line.split('=')
        row0=row[0]
        d[row0]=row[1].strip()
    return d

Overwriting src/mylib.py


dictionaty 형식을 가지는 함수명을 만들어라 d = dict()

f.readlines() 각각의 줄에대해서 

= 를 기준으로 split해라 

앞에있는건 키값 뒤에있는건 value값.

row[1].strip -> 뒤에 공백 제거

d[row0] -> 키값을 가지고 딕셔너리를 우선 만듬 

=row[1].strip() -> strip로 공백제거후 dictionary에 넣어줌 

결과는 키 value 구성이된 딕셔너리 열림

In [6]:
# import src.mylib
from src import mylib
#두가지 방식이 있다. 이번껀 src제거한 후 나오는 것
print (mylib.sayHello())


hello
None


현재 디렉토리 출력

In [7]:
import os
os.getcwd()

'/Users/nadonghyeon/BigData/201721402'

In [9]:
import os
import src.mylib

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
# 현재 디렉토리에 src와 key.properties까지 붙여서 절대경로로 만들어줌
key=src.mylib.getKey(keyPath)

In [10]:
print (key['dataseoul'])

73725974496a736c34395757705847


In [11]:
print (key['gokr'])

KeyError: 'gokr'

### 문제 2: 서울시 지하철역 정보 수집

In [3]:
import os
import src.mylib   # 상대경로

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties') # reading key saved in 'src'
key=src.mylib.getKey(keyPath)
KEY=str(key['dataseoul'])

keyPath로 만들고 

getKey getKey라는 함수로 받아온 결과를 key로 저장한다. 

key는 딕셔너리 타입으로 키와 value가 저장되어있다.

key['dataseoul'] -> 키를 읽어서 키의 value를 문자열로 저장후 KEY로 넣는다.




### 요청인자
서비스명 'SearchSTNBySubwayLineInfo'에 필요한 인자를 넣는다.

데이터 타입은 'json'

페이징에 필요한 INDEX를 조정한다.

호선 LINE_NUM을 설정한다.

In [4]:
# api
TYPE='json'
#SERVICE='SearchSTNBySubwayLineService'
SERVICE='SearchSTNBySubwayLineInfo'
START_INDEX=str(1)
END_INDEX=str(10)
LINE_NUM=str(2)

### url 만들기
앞서 설정한 서비스, 요청인자, 인증키를 슬래시("/")로 합쳐서 URL을 만들어보자.

In [5]:
params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,'','',LINE_NUM])


key는 비밀번호로서 30자리이다. 이러한 key는 보안상 출력에서 제외하는 것이 바람직 하겠다. key만 제외하고 출력해보자. 슬래시로 연결되어 있으니 0~29가 key에 해당하고, 30부터는 슬래시를 포함해 그 이후의 문자열을 출력한다.

In [6]:
print(params[30:])

/json/SearchSTNBySubwayLineInfo/1/10///2



urljoin은 url을 결합할 때 사용할 수 있다. 아래와 같이 url에 params을 덧붙인다. 문제는 url에서 슬래시가 제거된다.

In [7]:
import urllib
_url='http://openAPI.seoul.go.kr:8088/'
url=urllib.parse.urljoin(_url,params)
print(url) # backslashes removed

http://openAPI.seoul.go.kr:8088/656e705550736b6532324b4e4d715a/json/SearchSTNBySubwayLineInfo/1/10/2


urllib에 parse 함수를 통해 
urljoin함수 통해서 두개를 join을 한다.

### url 클릭해보면 출력이 안되기 때문에 

그래서 join() 함수로 문자열 합성을 한다. 주의할 점은 join() 함수는 리스트를 입력받는다.

In [8]:
_url='http://openAPI.seoul.go.kr:8088'
url="/".join([_url,params])
print(url) 

http://openAPI.seoul.go.kr:8088/656e705550736b6532324b4e4d715a/json/SearchSTNBySubwayLineInfo/1/10///2


### url 호출


In [9]:
import requests

r=requests.get(url)

In [10]:
stations=r.json()

In [11]:
type(stations)

dict

In [12]:
print (stations)

{'SearchSTNBySubwayLineInfo': {'list_total_count': 748, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'STATION_CD': '0159', 'STATION_NM': '동묘앞', 'STATION_NM_ENG': 'Dongmyo', 'LINE_NUM': '01호선', 'FR_CODE': '127'}, {'STATION_CD': '0416', 'STATION_NM': '미아사거리', 'STATION_NM_ENG': 'Miasamgeori', 'LINE_NUM': '04호선', 'FR_CODE': '416'}, {'STATION_CD': '0423', 'STATION_NM': '충무로', 'STATION_NM_ENG': 'Chungmuro', 'LINE_NUM': '04호선', 'FR_CODE': '423'}, {'STATION_CD': '1004', 'STATION_NM': '노량진', 'STATION_NM_ENG': 'Noryangjin', 'LINE_NUM': '01호선', 'FR_CODE': '136'}, {'STATION_CD': '1008', 'STATION_NM': '이촌', 'STATION_NM_ENG': 'Ichon', 'LINE_NUM': '경의선', 'FR_CODE': 'K111'}, {'STATION_CD': '1011', 'STATION_NM': '옥수', 'STATION_NM_ENG': 'Oksu', 'LINE_NUM': '경의선', 'FR_CODE': 'K114'}, {'STATION_CD': '1012', 'STATION_NM': '응봉', 'STATION_NM_ENG': 'Eungbong', 'LINE_NUM': '경의선', 'FR_CODE': 'K115'}, {'STATION_CD': '1013', 'STATION_NM': '왕십리', 'STATION_NM_ENG': 'Wangsimni', 'LINE_NUM': '경의선

위에는 json이다

### json 파싱

내가 원하는 데이터를 가져올려면 위의 데이터를 참고해서 
['SearchSTNBySubwayLineInfo']['row']안에 있다.
그렇게 access 해줘야 한다.

In [15]:
stations['SearchSTNBySubwayLineInfo']['row']

[{'STATION_CD': '0159',
  'STATION_NM': '동묘앞',
  'STATION_NM_ENG': 'Dongmyo',
  'LINE_NUM': '01호선',
  'FR_CODE': '127'},
 {'STATION_CD': '0416',
  'STATION_NM': '미아사거리',
  'STATION_NM_ENG': 'Miasamgeori',
  'LINE_NUM': '04호선',
  'FR_CODE': '416'},
 {'STATION_CD': '0423',
  'STATION_NM': '충무로',
  'STATION_NM_ENG': 'Chungmuro',
  'LINE_NUM': '04호선',
  'FR_CODE': '423'},
 {'STATION_CD': '1004',
  'STATION_NM': '노량진',
  'STATION_NM_ENG': 'Noryangjin',
  'LINE_NUM': '01호선',
  'FR_CODE': '136'},
 {'STATION_CD': '1008',
  'STATION_NM': '이촌',
  'STATION_NM_ENG': 'Ichon',
  'LINE_NUM': '경의선',
  'FR_CODE': 'K111'},
 {'STATION_CD': '1011',
  'STATION_NM': '옥수',
  'STATION_NM_ENG': 'Oksu',
  'LINE_NUM': '경의선',
  'FR_CODE': 'K114'},
 {'STATION_CD': '1012',
  'STATION_NM': '응봉',
  'STATION_NM_ENG': 'Eungbong',
  'LINE_NUM': '경의선',
  'FR_CODE': 'K115'},
 {'STATION_CD': '1013',
  'STATION_NM': '왕십리',
  'STATION_NM_ENG': 'Wangsimni',
  'LINE_NUM': '경의선',
  'FR_CODE': 'K116'},
 {'STATION_CD': '1015',
  

In [16]:
for e in stations['SearchSTNBySubwayLineInfo']['row']:
    print ("{0:4s}\t{1:15s}\t{2:3s}\t{3:2s}".format(e['STATION_CD'], e['STATION_NM'], e['FR_CODE'], e['LINE_NUM']))

0159	동묘앞            	127	01호선
0416	미아사거리          	416	04호선
0423	충무로            	423	04호선
1004	노량진            	136	01호선
1008	이촌             	K111	경의선
1011	옥수             	K114	경의선
1012	응봉             	K115	경의선
1013	왕십리            	K116	경의선
1015	회기             	123	01호선
1016	외대앞            	122	01호선


#### 일괄 실행

In [21]:
%%writefile src/ds_open_subway_json.py
#!/usr/bin/env python
# coding: utf-8
import os
import requests
import urllib
import mylib # NO! src.mylib

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=str(key['dataseoul'])
    TYPE='json'
    #OLD: SERVICE='SearchSTNBySubwayLineService'
    SERVICE='SearchSTNBySubwayLineInfo'
    LINE_NUM=str(2)
    START_INDEX=str(1)
    END_INDEX=str(10)
    #params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
    params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,'','',LINE_NUM])
    # (2) make a full url
    _url='http://openAPI.seoul.go.kr:8088' #NOTE slash: do not use 'http://openAPI.seoul.go.kr:8088/'
    #url=urllib.parse.urljoin(_url,params)
    url="/".join([_url,params])
    #print(url)
    # (3) get data
    r=requests.get(url)
    #print(r)
    stations=r.json()
    #print(stations)
    for e in stations['SearchSTNBySubwayLineInfo']['row']:
        print (u"{0:4s}\t{1:15s}\t{2:3s}\t{3:2s}".format(e['STATION_CD'], e['STATION_NM'], e['FR_CODE'], e['LINE_NUM']))

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_subway_json.py


 위  coding: utf-8는 코드가 깨질까봐 혹시나해서 써준것임

In [22]:
!python3 src/ds_open_subway_json.py

0159	동묘앞            	127	01호선
0416	미아사거리          	416	04호선
0423	충무로            	423	04호선
1004	노량진            	136	01호선
1008	이촌             	K111	경의선
1011	옥수             	K114	경의선
1012	응봉             	K115	경의선
1013	왕십리            	K116	경의선
1015	회기             	123	01호선
1016	외대앞            	122	01호선


### 결과가 없을 때까지 반복

# 한번 더 풀기

In [33]:
%%writefile src/ds_open_subway_iter_json.py
#!/usr/bin/env python
# coding: utf-8
import os
import requests
import urllib
import mylib # NO! src.mylib

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=str(key['dataseoul'])
    TYPE='json'
    #OLD: SERVICE='SearchSTNBySubwayLineService'
    SERVICE='SearchSTNBySubwayLineInfo'
    LINE_NUM=str(2)
    START_INDEX=str(1)
    END_INDEX=str(10)
    startIndex=1
    endIndex=10
    list_total_count=0    # set later
    while True:
        START_INDEX=str(startIndex)
        END_INDEX=str(endIndex)
       
        params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,'','',LINE_NUM])
        _url='http://openAPI.seoul.go.kr:8088' #NOTE slash: do not use 'http://openAPI.seoul.go.kr:8088/'
        url="/".join([_url,params])
        r=requests.get(url)
        stations=r.json()
        
        if(startIndex==1):
            list_total_count=stations['SearchSTNBySubwayLineInfo']['list_total_count']
            print("- Total Count: ", list_total_count)
        for e in stations['SearchSTNBySubwayLineInfo']['row']:
            print (u"{0:4s}\t{1:15s}\t{2:3s}\t{3:2s}".format(e['STATION_CD'], e['STATION_NM'], e['FR_CODE'], e['LINE_NUM']))
        startIndex+=10
        endIndex+=10
        if(endIndex > list_total_count):
            print("----- Ending endIndex=",endIndex)
            break  # exit from the while loop

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_subway_iter_json.py


In [34]:
!python3 src/ds_open_subway_iter_json.py

- Total Count:  748
0159	동묘앞            	127	01호선
0416	미아사거리          	416	04호선
0423	충무로            	423	04호선
1004	노량진            	136	01호선
1008	이촌             	K111	경의선
1011	옥수             	K114	경의선
1012	응봉             	K115	경의선
1013	왕십리            	K116	경의선
1015	회기             	123	01호선
1016	외대앞            	122	01호선
1017	신이문            	121	01호선
1018	석계             	120	01호선
1019	광운대            	119	01호선
101C	회기             	K118	경의선
1020	월계             	118	01호선
1021	녹천             	117	01호선
1022	창동             	116	01호선
1023	선릉             	K215	수인분당선
1025	도곡             	K217	수인분당선
1026	구룡             	K218	수인분당선
1027	개포동            	K219	수인분당선
1028	대모산입구          	K220	수인분당선
102C	왕십리            	K210	수인분당선
1030	수서             	K221	수인분당선
1031	복정             	K222	수인분당선
1032	신길             	138	01호선
1201	중랑             	K119	경의선
1202	상봉             	K120	경의선
1203	망우             	K121	경의선
1204	양원             	K122	경의선
1205	구리             	K123	경의선
1206	도농             	K124	경의선
1207

2516	발산             	515	05호선
2517	우장산            	516	05호선
2518	화곡             	517	05호선
2519	까치산            	518	05호선
2520	신정             	519	05호선
2521	목동             	520	05호선
2522	오목교            	521	05호선
2523	양평             	522	05호선
2524	영등포구청          	523	05호선
2525	영등포시장          	524	05호선
2526	신길             	525	05호선
2527	여의도            	526	05호선
2528	여의나루           	527	05호선
0220	선릉             	220	02호선
0238	합정             	238	02호선
0248	양천구청           	234-2	02호선
0317	경복궁            	327	03호선
2529	마포             	528	05호선
2530	공덕             	529	05호선
2531	애오개            	530	05호선
2532	충정로            	531	05호선
2533	서대문            	532	05호선
2534	광화문            	533	05호선
2535	종로3가           	534	05호선
2536	을지로4가          	535	05호선
2537	동대문역사문화공원      	536	05호선
2539	신금호            	538	05호선
2540	행당             	539	05호선
2541	왕십리            	540	05호선
2542	마장             	541	05호선
2544	장한평            	543	05호선
2545	군자             	544	05호선
2546	아차산            	545	05호선
2547	광나루

0236	영등포구청          	236	02호선
0240	신촌             	240	02호선
0241	이대             	241	02호선
0249	신정네거리          	234-3	02호선
0250	용두             	211-3	02호선
0312	불광             	322	03호선
0313	녹번             	323	03호선
0315	무악재            	325	03호선
0319	종로3가           	329	03호선
0320	을지로3가          	330	03호선
0324	금호             	334	03호선
0325	옥수             	335	03호선
0326	압구정            	336	03호선
4708	솔샘             	948	우이신설경전철
0208	왕십리            	208	02호선
0209	한양대            	209	02호선
0210	뚝섬             	210	02호선
0211	성수             	211	02호선
0212	건대입구           	212	02호선
0213	구의             	213	02호선
0214	강변             	214	02호선
0215	잠실나루           	215	02호선
0216	잠실             	216	02호선
0217	잠실새내           	217	02호선
0218	종합운동장          	218	02호선
0219	삼성             	219	02호선
0222	강남             	222	02호선
0223	교대             	223	02호선
0224	서초             	224	02호선
0225	방배             	225	02호선
0226	사당             	226	02호선
0227	낙성대            	227	02호선
0155	동대문            	128	01호선
015

### XML

In [35]:
import lxml
import lxml.etree
import requests

url='http://openAPI.seoul.go.kr:8088/73725974496a736c34395757705847/xml/SearchSTNBySubwayLineInfo/1/10///2'
data=requests.get(url).text
#requests 함수 가지고 text로 받아온다.
#tree=lxml.etree.parse(StringIO(data.encode('utf-8')))
tree=lxml.etree.fromstring(data.encode('utf-8'))

open data 에서 확인한 역의 이름 -> //STATION_NM 으로 역의 이름 가져올 수 있음
XPath를 이용한다. '//STATION_NM'는 어느 계층에서나 일치하는 태그를 검색한다.


In [36]:
for node in tree.xpath('//STATION_NM'):
    print (node.text)

동묘앞
미아사거리
충무로
노량진
이촌
옥수
응봉
왕십리
회기
외대앞


# 틀린것 다시풀기 리스트로 반환되기 때문에 반복문으로 처리

데이터의 갯수를 확인해보자. 리스트로 결과가 반환되기 때문에, 반복문으로 처리한다.

In [37]:
for node in tree.xpath('//list_total_count'):
    list_total_count=int(node.text)
    print ("total_count=",list_total_count, node.text)

total_count= 748 748


In [38]:
%%writefile src/ds_open_subway_xml.py
#!/usr/bin/env python
# coding: utf-8
import os
import requests
import urllib
import mylib # NO! src.mylib
#xml에 필요한 라이브러리들 아래
import lxml
import lxml.etree
from io import StringIO

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=str(key['dataseoul'])
    TYPE='xml'
    #OLD: SERVICE='SearchSTNBySubwayLineService'
    SERVICE='SearchSTNBySubwayLineInfo'
    LINE_NUM=str(2)
    START_INDEX=str(1)
    END_INDEX=str(10)
    startIndex=1
    endIndex=10
    list_total_count=0    # set later
    while True:
        START_INDEX=str(startIndex)
        END_INDEX=str(endIndex)
        #params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
        params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,'','',LINE_NUM])
        # (2) make a full url
        _url='http://openAPI.seoul.go.kr:8088' #NOTE slash: do not use 'http://openAPI.seoul.go.kr:8088/'
        #url=urllib.parse.urljoin(_url,params)
        url="/".join([_url,params])
        #print(url)
        # (3) get data
        data=requests.get(url).text
        #tree=lxml.etree.parse(StringIO(data.encode('utf-8')))
        tree=lxml.etree.fromstring(data.encode('utf-8'))
        if(startIndex==1):
            for node in tree.xpath('//list_total_count'):
                list_total_count=int(node.text)
                print ("- Total_count=",list_total_count)
        for node in tree.xpath('//STATION_NM'):
            print (node.text, end=", ")
            #end =',' 로 출력하는 문자를 엮는다. 
        startIndex+=10
        endIndex+=10
        if(endIndex > list_total_count):
            print("----- Ending endIndex=", endIndex)
            break

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_subway_xml.py


In [39]:
!python3 src/ds_open_subway_xml.py

- Total_count= 748
동묘앞, 미아사거리, 충무로, 노량진, 이촌, 옥수, 응봉, 왕십리, 회기, 외대앞, 신이문, 석계, 광운대, 회기, 월계, 녹천, 창동, 선릉, 도곡, 구룡, 개포동, 대모산입구, 왕십리, 수서, 복정, 신길, 중랑, 상봉, 망우, 양원, 구리, 도농, 양정, 덕소, 도심, 지축, 충무로, 매봉, 가락시장, 명동, 팔당, 운길산, 양수, 신원, 국수, 아신, 오빈, 양평, 용문, 지평, 서울역, 신촌, 효창공원앞, 공덕, 서강대, 홍대입구, 가좌, 디지털미디어시티, 강매, 행신, 능곡, 곡산, 백마, 풍산, 야당, 운정, 금릉, 금촌, 월롱, 문산, 임진강, 청량리, 중랑, 상봉, 서울대입구, 아현, 구파발, 대방, 망우, 신내, 별내, 퇴계원, 사릉, 금곡, 평내호평, 천마산, 마석, 대성리, 청평, 상천, 굴봉산, 백양리, 강촌, 김유정, 남춘천, 춘천, 봉명, 쌍용, 아산, 배방, 신창, 선바위, 경마공원, 대공원, 과천, 정부과천청사, 인덕원, 평촌, 금정, 판교, 이매, 삼동, 경기광주, 초월, 곤지암, 일원, 상계, 회현, 사당, 신둔도예촌, 부발, 구로, 가산디지털단지, 금천구청, 석수, 관악, 안양, 명학, 금정, 군포, 의왕, 성균관대, 화서, 수원, 독산, 세류, 병점, 세마, 오산대, 오산, 송탄, 평택지제, 평택, 성환, 직산, 두정, 천안, 서동탄, 광명, 산본, 대야미, 봉천, 잠원, 대치, 경찰병원, 반월, 상록수, 한대앞, 고잔, 초지, 안산, 신길온천, 정왕, 오이도, 수리산, 개봉, 오류동, 역곡, 부천, 송내, 부평, 백운, 동암, 주안, 제물포, 동인천, 인천, 구일, 소사, 부개, 간석, 도원, 온수, 중동, 한대앞, 초지, 안산, 신림, 용답, 노원, 성신여대입구, 신길온천, 정왕, 청량리, 수원, 서울숲, 압구정로데오, 강남구청, 선정릉, 가천대, 태평, 모란, 야탑, 서현, 수내, 정자, 미금, 오리, 이매, 보정, 구성, 기흥, 상갈, 영통, 망포, 매탄권선, 수원시청, 매교, 오

### 문제 3: 행정동별 서울생활인구 (단기체류 외국인)

656e705550736b6532324b4e4d715a


### 인증키

앞서 만들어 놓은 라이브러리를 사용하여 인증키를 읽어 온다.

In [ ]:
%%writefile src/key.properties
dataseoul = 656e705550736b6532324b4e4d715a


In [41]:
import os
import src.mylib
keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=src.mylib.getKey(keyPath)
KEY=str(key['dataseoul'])

### 요청인자


In [42]:

TYPE='xml'
SERVICE='SPOP_FORN_TEMP_RESD_DONG'
START_INDEX=str(1)
END_INDEX=str(5)
STDR_DE_ID=str(20200617)

#params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX)
params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STDR_DE_ID])

print (params[30:])

/xml/SPOP_FORN_TEMP_RESD_DONG/1/5/20200617


### url 만들기


In [43]:
import urllib
_url='http://openapi.seoul.go.kr:8088/'
url=urllib.parse.urljoin(_url,params)

In [44]:
print(url) 

http://openapi.seoul.go.kr:8088/73725974496a736c34395757705847/xml/SPOP_FORN_TEMP_RESD_DONG/1/5/20200617


## url 호출

In [45]:
import requests

data=requests.get(url).text

In [46]:
print (data)

<?xml version="1.0" encoding="UTF-8"?>
<SPOP_FORN_TEMP_RESD_DONG>
<list_total_count>9849</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110515</ADSTRD_CODE_SE>
<TOT_LVPOP_CO>248.866</TOT_LVPOP_CO>
<CHINA_STAYPOP_CO>86.4335</CHINA_STAYPOP_CO>
<ETC_STAYPOP_CO>162.432</ETC_STAYPOP_CO>
</row>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110530</ADSTRD_CODE_SE>
<TOT_LVPOP_CO>71.2628</TOT_LVPOP_CO>
<CHINA_STAYPOP_CO>0</CHINA_STAYPOP_CO>
<ETC_STAYPOP_CO>71.2627</ETC_STAYPOP_CO>
</row>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110540</ADSTRD_CODE_SE>
<TOT_LVPOP_CO>150.1777</TOT_LVPOP_CO>
<CHINA_STAYPOP_CO>86.2246</CHINA_STAYPOP_CO>
<ETC_STAYPOP_CO>63.9532</ETC_STAYPOP_CO>
</row>
<row>
<STDR_DE_ID>20200617</STDR_DE_ID>
<TMZON_PD_SE>00</TMZON_PD_SE>
<ADSTRD_CODE_SE>11110550</ADST

### XML 파싱
TOT_LVPOP_CO 항목을 조회해보자.

In [47]:
import lxml
import lxml.etree

tree=lxml.etree.fromstring(data.encode('utf-8'))
nodes=tree.xpath('//TOT_LVPOP_CO')
for node in nodes:
    print (node.text)

248.866
71.2628
150.1777
136.7137
174.824


lxml.etree.fromstring(data.encode('utf-8')) -> data를 인코딩해야함 안해도 되는데 그냥 해라 

xpath 이용해서 특정 태그를 출력해라 

### 일괄실행
* XML 데이터를 처리하는 프로그램이다.
* 앞 문제풀이 프로그램과 큰 차이가 없다. API에서 읽는 방식이 유사하다.

In [64]:
%%writefile src/ds_open_foreigners_xml.py
import os
import urllib
import requests
import lxml
import lxml.etree
#import mylib #from src import mylib
# 현재 호출하는 mylib이 src밑에 있고 파일을 읽을때 위에서 이미 src를 했기 때문에
# from src import mylib안해도된다. 

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    #syntax에러로 인하여 mylib사용안하고 키를 직접 넣어줬다.
    # key=mylib.getKey(keyPath)
    # KEY=str(key['dataseoul'])
    TYPE='xml'
    SERVICE='SPOP_FORN_TEMP_RESD_DONG'
    START_INDEX=str(1)
    END_INDEX=str(10)
    STDR_DE_ID=str(20200617)
    #params="/".join([KEY,TYPE,SERVICE,START_INDEX,END_INDEX,STDR_DE_ID])

    #_url='http://openAPI.seoul.go.kr:8088/'
    _url='http://openAPI.seoul.go.kr:8088' # NOTE: the slash at the end removed
    #url=urllib.parse.urljoin(_url,params)
    url="/".join([_url,"656e705550736b6532324b4e4d715a",TYPE,SERVICE,START_INDEX,END_INDEX,STDR_DE_ID])
    data=requests.get(url).text
    #tree=lxml.etree.parse(StringIO.StringIO(data.encode('utf-8')))
    tree=lxml.etree.fromstring(data.encode('utf-8'))
    nodes=tree.xpath('//TOT_LVPOP_CO')
    for node in nodes:
        print (node.text)

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_foreigners_xml.py


In [65]:

!python3 src/ds_open_foreigners_xml.py

248.866
71.2628
150.1777
136.7137
174.824
0.4833
0.2163
35.2093
756.5433
87.6117


### 문제 4: 서울시 지하철역별 월별 승하차인원 구하기
'서울시 지하철호선별 역별 승하차 인원 정보'를 API를 통해 데이터를 수집해보자. 서울열린데이터광장에서 '지하철 승하차인원'을 찾아보면, 'CardSubwayStatsNew' API가 존재한다.



### 인증키

564165694a736b6538305678554851

In [40]:
_url='http://openAPI.seoul.go.kr:8088'
#_key=str(key['dataseoul'])
_key="564165694a736b6538305678554851"
_type='json'
_service='CardSubwayStatsNew'
_start_index=1

_end_index=5
_use_dt='20210801'

### url 만들기

In [41]:
_api=os.path.join(_url,_key,_type,_service,str(_start_index),str(_end_index),_use_dt)
print(_api)

http://openAPI.seoul.go.kr:8088/564165694a736b6538305678554851/json/CardSubwayStatsNew/1/5/20210801


### url 호출

In [35]:
import requests
response = requests.get(_api)

## json 파싱

string 문자를 그냥 가져올떄는 

.text했는데 

json 파일 호출 할때는 .json()이라고 함

response

In [36]:

sub=response.json()

In [37]:
type(sub)

dict

In [38]:
print(sub)

{'CardSubwayStatsNew': {'list_total_count': 600, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'USE_DT': '20210801', 'LINE_NUM': '1호선', 'SUB_STA_NM': '서울역', 'RIDE_PASGR_NUM': 17896.0, 'ALIGHT_PASGR_NUM': 15468.0, 'WORK_DT': '20210804'}, {'USE_DT': '20210801', 'LINE_NUM': '1호선', 'SUB_STA_NM': '동묘앞', 'RIDE_PASGR_NUM': 7760.0, 'ALIGHT_PASGR_NUM': 7771.0, 'WORK_DT': '20210804'}, {'USE_DT': '20210801', 'LINE_NUM': '1호선', 'SUB_STA_NM': '시청', 'RIDE_PASGR_NUM': 4436.0, 'ALIGHT_PASGR_NUM': 4308.0, 'WORK_DT': '20210804'}, {'USE_DT': '20210801', 'LINE_NUM': '1호선', 'SUB_STA_NM': '종각', 'RIDE_PASGR_NUM': 7972.0, 'ALIGHT_PASGR_NUM': 7438.0, 'WORK_DT': '20210804'}, {'USE_DT': '20210801', 'LINE_NUM': '1호선', 'SUB_STA_NM': '종로3가', 'RIDE_PASGR_NUM': 8999.0, 'ALIGHT_PASGR_NUM': 8604.0, 'WORK_DT': '20210804'}]}}


일부만 읽을 때는 괄호를 계층적으로 따라가라


In [71]:
sub['CardSubwayStatsNew']['row']

[{'USE_DT': '20210801',
  'LINE_NUM': '1호선',
  'SUB_STA_NM': '서울역',
  'RIDE_PASGR_NUM': 17896.0,
  'ALIGHT_PASGR_NUM': 15468.0,
  'WORK_DT': '20210804'},
 {'USE_DT': '20210801',
  'LINE_NUM': '1호선',
  'SUB_STA_NM': '동묘앞',
  'RIDE_PASGR_NUM': 7760.0,
  'ALIGHT_PASGR_NUM': 7771.0,
  'WORK_DT': '20210804'},
 {'USE_DT': '20210801',
  'LINE_NUM': '1호선',
  'SUB_STA_NM': '시청',
  'RIDE_PASGR_NUM': 4436.0,
  'ALIGHT_PASGR_NUM': 4308.0,
  'WORK_DT': '20210804'},
 {'USE_DT': '20210801',
  'LINE_NUM': '1호선',
  'SUB_STA_NM': '종각',
  'RIDE_PASGR_NUM': 7972.0,
  'ALIGHT_PASGR_NUM': 7438.0,
  'WORK_DT': '20210804'},
 {'USE_DT': '20210801',
  'LINE_NUM': '1호선',
  'SUB_STA_NM': '종로3가',
  'RIDE_PASGR_NUM': 8999.0,
  'ALIGHT_PASGR_NUM': 8604.0,
  'WORK_DT': '20210804'}]

In [91]:
a = [{
    'dd': '1',
    'aa': '2'
},
{
    'dd': '1',
    'aa': '2'
},
{
    'dd': '1',
    'aa': '2'
}]

for e in a:
    print("{0:10s}\t{1:10s}".format(e['dd'], e['aa']))

1         	2         
1         	2         
1         	2         


In [86]:
for e in sub['CardSubwayStatsNew']['row']:
    print ("{0:5s}\t{1:10s}\t{2:9.1f}\t{3:9.1f}".format(e['LINE_NUM'],e['SUB_STA_NM'],e['RIDE_PASGR_NUM'],e['ALIGHT_PASGR_NUM']))

1호선  	서울역       	  17896.0	  15468.0
1호선  	동묘앞       	   7760.0	   7771.0
1호선  	시청        	   4436.0	   4308.0
1호선  	종각        	   7972.0	   7438.0
1호선  	종로3가      	   8999.0	   8604.0


0:5s -> 5s는 문자열로 5자리를 출력하는 것이다.

1:10s -> 10자리로 문자를 출력해라

2:9.1f -> float로 소수점 한자리를 출력하라 
## 이 print함수 다시 보자리스트 안에 딕셔너리가 있을때 사용하는 것이다.

## 문제 5: 서울시 지하철 호선별 역별 시간대별 승하차 인원 정보¶


지하철역의 승차, 하차 인원을 알아보자. 서울열린데이터광장을 '지하철 승하차'를 찾아보면, 몇 가지 API가 있다. 이 가운데 '서울시 지하철 호선별 역별 시간대별 승하차 인원 정보' 'CardSubwayTime' API를 통해 데이터를 수집해보자. 교통카드(선후불교통카드 및 1회용 교통카드)를 이용한 지하철 호선별 역별(1~9호선, 서울시 관할 운송기관에 한함) 시간대별 승하차인원을 알아볼 수 있다. 데이터는 매월 5일 전월 데이터를 갱신된다.

46416e4957736b653132314d445a6347

In [47]:
_url='http://openAPI.seoul.go.kr:8088'
_key="46416e4957736b653132314d445a6347"
_type='xml'
_service='CardSubwayTime'
_start_index=1
_end_index=5
_use_mon='202106'


### url 호출 반복
아래 출력을 살펴보면, RESULT, row 태그가 포함되어 있다.

API호출에 대한 처리가 정상적인지 알 수 있다.
메시지	메시지 설명

INFO-000	정상 처리되었습니다.

INFO-100	인증키가 유효하지 않습니다.

INFO-200	해당하는 데이터가 없습니다.

row 태그 안에는 요청 정보가 포함되어 있다.

여기서는 INDEX를 통해 1회에 5건씩, iter를 통해 1회 반복해서 총 5건의 데이터를 가져오고 있다. iter를 크게 또는 작게 해서 데이터 수집량을 조정할 수 있다. 각 레코드는 시간대별 승하차인원을 포함하고 있다.


# 틀림 다시풀어

In [45]:
import requests
# 초기 iteration은 0부터 시작한다.
# _maxiter만큼 반복한다. 1하면 1회 2하면 2회 


_maxIter=2
_iter=0

while _iter<_maxIter:
    _api="/".join([_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon])
    #print _api
    response = requests.get(_api).text
    print(response)
    _start_index+=5
    _end_index+=5
    _iter+=1

<?xml version="1.0" encoding="UTF-8"?>
<CardSubwayTime>
<list_total_count>608</list_total_count>
<RESULT>
<CODE>INFO-000</CODE>
<MESSAGE>정상 처리되었습니다</MESSAGE>
</RESULT>
<row>
<USE_MON>202106</USE_MON>
<LINE_NUM>1호선</LINE_NUM>
<SUB_STA_NM>서울역</SUB_STA_NM>
<FOUR_RIDE_NUM>654</FOUR_RIDE_NUM>
<FOUR_ALIGHT_NUM>17</FOUR_ALIGHT_NUM>
<FIVE_RIDE_NUM>9008</FIVE_RIDE_NUM>
<FIVE_ALIGHT_NUM>6400</FIVE_ALIGHT_NUM>
<SIX_RIDE_NUM>12474</SIX_RIDE_NUM>
<SIX_ALIGHT_NUM>37203</SIX_ALIGHT_NUM>
<SEVEN_RIDE_NUM>37253</SEVEN_RIDE_NUM>
<SEVEN_ALIGHT_NUM>91875</SEVEN_ALIGHT_NUM>
<EIGHT_RIDE_NUM>59876</EIGHT_RIDE_NUM>
<EIGHT_ALIGHT_NUM>187805</EIGHT_ALIGHT_NUM>
<NINE_RIDE_NUM>44619</NINE_RIDE_NUM>
<NINE_ALIGHT_NUM>118679</NINE_ALIGHT_NUM>
<TEN_RIDE_NUM>42611</TEN_RIDE_NUM>
<TEN_ALIGHT_NUM>57710</TEN_ALIGHT_NUM>
<ELEVEN_RIDE_NUM>49533</ELEVEN_RIDE_NUM>
<ELEVEN_ALIGHT_NUM>50003</ELEVEN_ALIGHT_NUM>
<TWELVE_RIDE_NUM>59357</TWELVE_RIDE_NUM>
<TWELVE_ALIGHT_NUM>53317</TWELVE_ALIGHT_NUM>
<THIRTEEN_RIDE_NUM>61171</THIRTEE

### 위 예제의 json 반환

이번에는 json으로 반환해보자.

In [98]:
_type='json'

In [99]:
_api="/".join([_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon])

In [100]:
data=requests.get(_api).text
print(data)

{"CardSubwayTime":{"list_total_count":608,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"USE_MON":"202106","LINE_NUM":"2호선","SUB_STA_NM":"신답","FOUR_RIDE_NUM":4.0,"FOUR_ALIGHT_NUM":1.0,"FIVE_RIDE_NUM":1052.0,"FIVE_ALIGHT_NUM":100.0,"SIX_RIDE_NUM":1755.0,"SIX_ALIGHT_NUM":487.0,"SEVEN_RIDE_NUM":6550.0,"SEVEN_ALIGHT_NUM":890.0,"EIGHT_RIDE_NUM":9858.0,"EIGHT_ALIGHT_NUM":1699.0,"NINE_RIDE_NUM":4205.0,"NINE_ALIGHT_NUM":1080.0,"TEN_RIDE_NUM":2180.0,"TEN_ALIGHT_NUM":1003.0,"ELEVEN_RIDE_NUM":1950.0,"ELEVEN_ALIGHT_NUM":1173.0,"TWELVE_RIDE_NUM":1808.0,"TWELVE_ALIGHT_NUM":1344.0,"THIRTEEN_RIDE_NUM":1767.0,"THIRTEEN_ALIGHT_NUM":1436.0,"FOURTEEN_RIDE_NUM":1594.0,"FOURTEEN_ALIGHT_NUM":1647.0,"FIFTEEN_RIDE_NUM":1488.0,"FIFTEEN_ALIGHT_NUM":2382.0,"SIXTEEN_RIDE_NUM":1682.0,"SIXTEEN_ALIGHT_NUM":3189.0,"SEVENTEEN_RIDE_NUM":1769.0,"SEVENTEEN_ALIGHT_NUM":4230.0,"EIGHTEEN_RIDE_NUM":2233.0,"EIGHTEEN_ALIGHT_NUM":9703.0,"NINETEEN_RIDE_NUM":976.0,"NINETEEN_ALIGHT_NUM":6552.0,"TWENTY_RIDE_NUM":589.0,

## json 파싱

json.loads() 함수는 문자열을 json으로 변환해준다.

In [39]:
import json
jsubwayTime = json.loads(data)

NameError: name 'data' is not defined

In [104]:
j0=jsubwayTime['CardSubwayTime']['row'][0]


키값을 읽어라 

In [105]:
j0.keys()

dict_keys(['USE_MON', 'LINE_NUM', 'SUB_STA_NM', 'FOUR_RIDE_NUM', 'FOUR_ALIGHT_NUM', 'FIVE_RIDE_NUM', 'FIVE_ALIGHT_NUM', 'SIX_RIDE_NUM', 'SIX_ALIGHT_NUM', 'SEVEN_RIDE_NUM', 'SEVEN_ALIGHT_NUM', 'EIGHT_RIDE_NUM', 'EIGHT_ALIGHT_NUM', 'NINE_RIDE_NUM', 'NINE_ALIGHT_NUM', 'TEN_RIDE_NUM', 'TEN_ALIGHT_NUM', 'ELEVEN_RIDE_NUM', 'ELEVEN_ALIGHT_NUM', 'TWELVE_RIDE_NUM', 'TWELVE_ALIGHT_NUM', 'THIRTEEN_RIDE_NUM', 'THIRTEEN_ALIGHT_NUM', 'FOURTEEN_RIDE_NUM', 'FOURTEEN_ALIGHT_NUM', 'FIFTEEN_RIDE_NUM', 'FIFTEEN_ALIGHT_NUM', 'SIXTEEN_RIDE_NUM', 'SIXTEEN_ALIGHT_NUM', 'SEVENTEEN_RIDE_NUM', 'SEVENTEEN_ALIGHT_NUM', 'EIGHTEEN_RIDE_NUM', 'EIGHTEEN_ALIGHT_NUM', 'NINETEEN_RIDE_NUM', 'NINETEEN_ALIGHT_NUM', 'TWENTY_RIDE_NUM', 'TWENTY_ALIGHT_NUM', 'TWENTY_ONE_RIDE_NUM', 'TWENTY_ONE_ALIGHT_NUM', 'TWENTY_TWO_RIDE_NUM', 'TWENTY_TWO_ALIGHT_NUM', 'TWENTY_THREE_RIDE_NUM', 'TWENTY_THREE_ALIGHT_NUM', 'MIDNIGHT_RIDE_NUM', 'MIDNIGHT_ALIGHT_NUM', 'ONE_RIDE_NUM', 'ONE_ALIGHT_NUM', 'TWO_RIDE_NUM', 'TWO_ALIGHT_NUM', 'THREE_RIDE_NU

sub_sta_nm을 읽어라

In [107]:
j0.get('SUB_STA_NM')

'신답'

In [108]:
for item in jsubwayTime['CardSubwayTime']['row']:
    print(item.values())

dict_values(['202106', '2호선', '신답', 4.0, 1.0, 1052.0, 100.0, 1755.0, 487.0, 6550.0, 890.0, 9858.0, 1699.0, 4205.0, 1080.0, 2180.0, 1003.0, 1950.0, 1173.0, 1808.0, 1344.0, 1767.0, 1436.0, 1594.0, 1647.0, 1488.0, 2382.0, 1682.0, 3189.0, 1769.0, 4230.0, 2233.0, 9703.0, 976.0, 6552.0, 589.0, 3222.0, 546.0, 3377.0, 534.0, 3394.0, 87.0, 1709.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '20210703'])
dict_values(['202106', '2호선', '신설동', 35.0, 0.0, 1067.0, 383.0, 1310.0, 2273.0, 3601.0, 5566.0, 6334.0, 13082.0, 4054.0, 8209.0, 3804.0, 5939.0, 4649.0, 6264.0, 5669.0, 6076.0, 6277.0, 6188.0, 6827.0, 5461.0, 7516.0, 5089.0, 7728.0, 4245.0, 10802.0, 4796.0, 10839.0, 5544.0, 4287.0, 3970.0, 2653.0, 2287.0, 2313.0, 2061.0, 1738.0, 2598.0, 397.0, 1208.0, 0.0, 239.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, '20210703'])
dict_values(['202106', '2호선', '도림천', 0.0, 0.0, 263.0, 320.0, 1272.0, 833.0, 3182.0, 4678.0, 3286.0, 6326.0, 2138.0, 2308.0, 1836.0, 1521.0, 1620.0, 1305.0, 2489.0, 1941.0, 1670.0, 1452.0, 2297.0, 18

## 하나의 프로그램으로

## 틀림 다시해라 

In [109]:
import os
from src import mylib
import requests

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=mylib.getKey(keyPath)

_url='http://openAPI.seoul.go.kr:8088'
_key=str(key['dataseoul'])
_type='json'
_service='CardSubwayTime'
_start_index=1
_end_index=5
_use_mon='202106'

_maxIter=1
_iter=0
while _iter<_maxIter:
    _api="/".join([_url,_key,_type,_service,str(_start_index),str(_end_index),_use_mon])
    #print _api
    subwayTime = requests.get(_api).text
    print(subwayTime)
    _start_index+=5
    _end_index+=5
    _iter+=1

{"CardSubwayTime":{"list_total_count":608,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"USE_MON":"202106","LINE_NUM":"1호선","SUB_STA_NM":"서울역","FOUR_RIDE_NUM":654.0,"FOUR_ALIGHT_NUM":17.0,"FIVE_RIDE_NUM":9008.0,"FIVE_ALIGHT_NUM":6400.0,"SIX_RIDE_NUM":12474.0,"SIX_ALIGHT_NUM":37203.0,"SEVEN_RIDE_NUM":37253.0,"SEVEN_ALIGHT_NUM":91875.0,"EIGHT_RIDE_NUM":59876.0,"EIGHT_ALIGHT_NUM":187805.0,"NINE_RIDE_NUM":44619.0,"NINE_ALIGHT_NUM":118679.0,"TEN_RIDE_NUM":42611.0,"TEN_ALIGHT_NUM":57710.0,"ELEVEN_RIDE_NUM":49533.0,"ELEVEN_ALIGHT_NUM":50003.0,"TWELVE_RIDE_NUM":59357.0,"TWELVE_ALIGHT_NUM":53317.0,"THIRTEEN_RIDE_NUM":61171.0,"THIRTEEN_ALIGHT_NUM":53687.0,"FOURTEEN_RIDE_NUM":53310.0,"FOURTEEN_ALIGHT_NUM":49094.0,"FIFTEEN_RIDE_NUM":65767.0,"FIFTEEN_ALIGHT_NUM":52788.0,"SIXTEEN_RIDE_NUM":76249.0,"SIXTEEN_ALIGHT_NUM":53969.0,"SEVENTEEN_RIDE_NUM":122928.0,"SEVENTEEN_ALIGHT_NUM":64693.0,"EIGHTEEN_RIDE_NUM":184907.0,"EIGHTEEN_ALIGHT_NUM":73978.0,"NINETEEN_RIDE_NUM":87575.0,"NINETEEN_ALIG